<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/NLP_pytorch/NLP_ch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파이토치로 배우는 자연어 처리: ch1
## 1. 소개   
1.1 지도학습   
1.2 샘플과 타깃의 인코딩   
  - 원-핫 인코딩, TF 표현, TF-IDF 표현, 타깃 인코딩   

1.3 계산 그래프   
1.4 파이토치 기초
  - 파이토치 설치, 텐서 만들기, 텐서 타입과 크기, 텐서, 텐서와 계산 그래프, CUDA 텐서

---


## 1.1 지도학습
- observation에 대응하는 target의 정답을 제공하는 방식   
주요 개념   
- `샘플`: 예측에 사용하는 아이템, 입력(input) / x
- `타깃`: 샘플에 상응하는 레이블(label) / 예측되는 대상 / 정답, y
- `모델(model)`: 수학식이나 샘플x를 받아 타깃 레이블값을 예측하는 함수
- `parameter`: 가중치(weight) / 모델을 규정함 / w
- `예측(prediction)`: 모델이 추축하는 target값 / 추정(estimate) / hat 표기
- `손실함수(loss function)`: 훈련 데이터에 대한 예측이 target과 얼마나 차이 나는지 비교하는 함수 /  L

#### gradient descent
- data set에서 손실함수를 최소화하는 parameter 값 찾기
- stochastic gradient descent(SGD): 확률적 경사 하강법
  - data point를 하나 또는 일부 랜덤하게 선택하여 gradient 계산
- minibatch SGD: pure SGD 여러 개 사용하는 방법
- pure SGD: updata 잡음이 많아 수렴이 매우 느림
- backpropagation: 역전파, parameter를 반복적으로 update하는 과정
    - 각 단계(epoch)는 정방향계산(foward pass), 역방향 계산(backward padd) 구성
    - foward: 현재 parameter 값으로 입력을 평가해 손실함수 계산
    - backward: 손실의 gradient를 사용해 parameter update
---






## 1.2 샘플과 타깃의 인코딩



### 1.2.1. one-hot representation
- 0 벡터에서 시작해 등장하는 단어 원소를 1로 설정   
example) Time files like an arrow. / Fruit flies like a banana.
- 문장을 토큰(token)으로 나누고 구두점을 무시하고 모두 소문자로 바꾸면 어휘사전(vocabulary)의 크기는 8이 된다.
- { time, fruit, flies, like, a , an, arrow, banana}
- "like a banana"의 one-hot 표현은 3X8 행렬이 됩니다. 행은 8차원 one-hot 벡터 



### 1.2.2 TF 표현(문서 빈도)
- 단순히 소속 단어의 one-hot representation을 합해 만듦   
- example) Fruit flies like time flies a fruit -> [1, 2, 2, 1, 1, 0, 0, 0] 각 원소는 해당 단어가 문장에 등장하는 횟수
- `CountVectorizer` 클래스에서 binary=True로 지정해서 원핫인코딩으로 변환
  - 기본값은 false로 단어 등장 횟수를 기록한 TF표현을 만든다.
  - 기본적으로 글자 하나로 이루어진 단어를 무시
  - 희소행렬을 반환하므로 toarray()메서드를 사용해 밀집벡터형태로 바꿔 출력
 


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns

corpus =['Time flies like an arrow.', 
         'Fruit flie like a banana.']
one_hot_vectorizer = CountVectorizer(binary=True) #  binary=True로 지정해서 원핫인코딩으로 변환, 기본값은 false로 단어 등장 횟수를 기록한 TF표현을 만든다.
one_hot = one_hot_vectorizer.fit_transform(corpus).toarray()
one_hot = one_hot_vectorizer.get_feature_names()
sns.heatmap(one_hot, annot=True, 
            cbar=False, xticklabels=vocab,
            yticklabels=['Sentence 1', 'Sentence 2'])

### 1.2.3 TF-IDF 표현
- TF: 등장 횟수에 비례하여 단어에 가중치 부여, 흔한 단어에는 X
- inverse-document-frequency(IDF): 흔한 토큰의 점수를 낮추고 드문 토큰의 점수를 높임
- 딥러닝의 목적은 표현 학습이므로 보통 TF-IDF 같이 경험적인 방법으로 입력을 인코딩하지 않는다. 



### 1.2.4 타깃 인코딩
많은 NLP 작업은 범주형 레이블을 사용
- 모델은 고정된 한 세트의 레이블 중 하나를 예측해야 함 이를 인코딩할 때는 레이블마다 고유한 인덱스를 부여하는 방법을 가정 많이 사용, 하지만 이런 간단한 표현은 출력 레이블 수가 너무 커지면 문제가 된다.ex) 언어 모델링: 이전 단어로 다음 단어 예측



---

## 1.3 계산 그래프

---


## 1.4 파이토치 기초
- `파이토치`: 딥러닝용 패키지를 제공하는 최적화된 텐서 조작 라이브러리
- `텐서`: 다차원 데이터를 담은 수학 객체

#### 1.4.2 텐서 만들기

In [3]:
import torch
import numpy as np

In [4]:
def describe(x):
    print("타입: {}".format(x.type()))
    print("크기: {}".format(x.shape))
    print("값: \n{}".format(x))

In [6]:
# torch.Tensor로 텐서 만들기
describe(torch.Tensor(2, 3))

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[1.9133e+09, 3.0659e-41, 3.7835e-44],
        [0.0000e+00,        nan, 0.0000e+00]])


[0, 1) 범위의 균등 분포에서 샘플링한 값으로 랜덤하게 초기화

In [ ]:
# 특정 크기의 랜덤한 텐서를 만드느 것이 일반적입니다.
t = torch.rand(2, 3)  # 균등 분포
t = torch.randn(2, 3) # 표준 균등 분포

동일한 스칼라값으로 채운 텐서   
`zeros()`, `ones()`, `fill_()`   
- `in-place`: (_)가 있는 메서드는 텐서값을 바꾸는 연산
  - 새로운 객체를 만들지 않고 현재값을 변경
- 정규분포 쓰는 `normal_()`, 균등분포 쓰는 `uniform_()`

In [8]:
describe(torch.zeros(2, 3))
x = torch.ones(2, 3)
describe(x)
x.fill_(5)
describe(x)

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[0., 0., 0.],
        [0., 0., 0.]])
타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[1., 1., 1.],
        [1., 1., 1.]])
타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[5., 5., 5.],
        [5., 5., 5.]])


파이썬 리스트로 텐서 만들기

In [11]:
x = torch.Tensor([[1, 2,],  
                  [2, 4,]])
describe(x)

타입: torch.FloatTensor
크기: torch.Size([2, 2])
값: 
tensor([[1., 2.],
        [2., 4.]])


numpy로 텐서 만들고 초기화
- 언제든지 텐서를 numpy 배열로 변경 가능
- numpy 배열은 tensor type이 FloatTensor가 아니라 DoubleTensor가 됨

In [12]:
import numpy as np

npy = np.random.rand(2, 3)
describe(torch.from_numpy(npy))
print(npy.dtype)

타입: torch.DoubleTensor
크기: torch.Size([2, 3])
값: 
tensor([[0.8947, 0.2249, 0.0415],
        [0.0341, 0.7607, 0.9446]], dtype=torch.float64)
float64


#### 1.4.3 텐서 타입과 크기
- torch.Tensor 생성자를 사용할 째 기본 텐서 타입은 torch.FloatTensor
- `타입 캐스팅`을 사용해 타입 변경 가능
- `특정 텐서 타입 생성자 직접 호출`, `dtype` 사용
- `shape`니 `size()`로 텐서의 차원 확인


#### 1.4.4 텐서 연산
- `view` 메서드를 사용하면 원소의 순서를 유지하면서 텐서의 차원을 자유롭게 바꿀 수 있습니다.

In [24]:
x = torch.arange(0, 20)

print(x.view(1, 20))
print(x.view(2, 10))
print(x.view(4, 5))

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19]])
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]])
tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])


- 뷰를 사용하여 크기가 1인 차원을 추가할 수 있습니다. 이렇게 하면 다른 텐서와 연산할 때 브로드캐스팅을 활용할 수 있습니다.

In [26]:
x = torch.arange(12).view(3, 4)
y = torch.arange(4).view(1, 4)

print(x)
print(y)
print(x + y)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
tensor([[0, 1, 2, 3]])
tensor([[ 0,  2,  4,  6],
        [ 4,  6,  8, 10],
        [ 8, 10, 12, 14]])


- unsqueeze와 squeeze는 크기가 1인 차원을 추가하고 삭제합니다.

In [27]:
x = torch.arange(12).view(3, 4)
print(x.shape)

x = x.unsqueeze(dim=1)
print(x.shape)

x = x.squeeze()
print(x.shape)

torch.Size([3, 4])
torch.Size([3, 1, 4])
torch.Size([3, 4])


- 표준 수학 연산을 모두 지원합니다(예를 들어 add).

In [28]:
x = torch.randn(2, 3)
describe(x)

describe(torch.add(x, x))

describe(x + x)
# 인플레이스 연산
print(x.add_(x))

타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[-1.3670, -0.6722, -1.6764],
        [ 0.8515,  1.3238,  0.5128]])
타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[-2.7340, -1.3445, -3.3529],
        [ 1.7029,  2.6475,  1.0255]])
타입: torch.FloatTensor
크기: torch.Size([2, 3])
값: 
tensor([[-2.7340, -1.3445, -3.3529],
        [ 1.7029,  2.6475,  1.0255]])
tensor([[-2.7340, -1.3445, -3.3529],
        [ 1.7029,  2.6475,  1.0255]])


- 차원을 줄이는 연산이 많이 있습니다. 예를 들면 `sum`입니다.

In [29]:
x = torch.arange(12).reshape(3, 4)
print("x: \n", x)
print("---")
print("행을 따라 덧셈 (dim=0): \n", x.sum(dim=0))
print("---")
print("열을 따라 덧셈 (dim=1): \n", x.sum(dim=1))

x: 
 tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
---
행을 따라 덧셈 (dim=0): 
 tensor([12, 15, 18, 21])
---
열을 따라 덧셈 (dim=1): 
 tensor([ 6, 22, 38])


#### 1.4.5 인덱싱, 슬라이싱, 연결

In [21]:
x = torch.arange(6).view(2, 3)
print("x: \n", x)
print("---")
print("x[:2, :2]: \n", x[:2, :2])
print("---")
print("x[0][1]: \n", x[0][1])
print("---")
print("[0][1]에 8을 할당")
x[0][1] = 8
print(x)

x: 
 tensor([[0, 1, 2],
        [3, 4, 5]])
---
x[:2, :2]: 
 tensor([[0, 1],
        [3, 4]])
---
x[0][1]: 
 tensor(1)
---
[0][1]에 8을 할당
tensor([[0, 8, 2],
        [3, 4, 5]])


- index_select을 사용해 텐서의 원소를 선택할 수 있습니다.

In [30]:
x = torch.arange(9).view(3,3)
print(x)

print("---")
indices = torch.LongTensor([0, 2])
print(torch.index_select(x, dim=0, index=indices))

print("---")
indices = torch.LongTensor([0, 2])
print(torch.index_select(x, dim=1, index=indices))

tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
---
tensor([[0, 1, 2],
        [6, 7, 8]])
---
tensor([[0, 2],
        [3, 5],
        [6, 8]])


- 텐서 연결

In [ ]:
torch.cat([x, x], dim=0)

tensor([[0, 1, 2],
        [3, 4, 5],
        [0, 1, 2],
        [3, 4, 5]])

In [ ]:
torch.cat([x, x], dim=1)

tensor([[0, 1, 2, 0, 1, 2],
        [3, 4, 5, 3, 4, 5]])

In [ ]:
torch.stack([x, x])

tensor([[[0, 1, 2],
         [3, 4, 5]],

        [[0, 1, 2],
         [3, 4, 5]]])

- 선형 대수 연산
- 전치

In [31]:
x = torch.arange(0, 12).view(3,4)
print("x: \n", x) 
print("---")
print("x.tranpose(1, 0): \n", x.transpose(1, 0))

x: 
 tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
---
x.tranpose(1, 0): 
 tensor([[ 0,  4,  8],
        [ 1,  5,  9],
        [ 2,  6, 10],
        [ 3,  7, 11]])


In [ ]:
torch.mm(x1, x2)

tensor([[ 3.,  6.],
        [12., 24.]])

#### 1.4.6 텐서와 계산그래프
- requires_grad = True로 지정하면 gradient 기반 학습에 필요한 손실 함수와 tensor의 gradient를 기록하는 부가 연산을 활성화한다.

In [ ]:
x = torch.ones(2, 2, requires_grad= True)
print(x)
print(x.grad is None)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
True


In [ ]:
y = (x+2) * (x+5) +3
print(y)
print(x.grad is None)

tensor([[21., 21.],
        [21., 21.]], grad_fn=<AddBackward0>)
True


In [ ]:
z = y.mean()
print(z)
z.backward()
print(x.grad is None)

tensor(21., grad_fn=<MeanBackward0>)
False


#### 1.4.7 CUDA tensor
- 지금까지 tensor를 CPU 메모리에 할당함, CPU가 있다면 선형대수연산을 수행할 떄 되도록 사용 권장
- GPU를 사용하려면 먼저 tensor를 GPU에 할당해야 함
- CUDA API를 사용해서 GPU 활용 가능: NVIDIA GPU에서만 사용
- torch.cuda.is_available()로 GPU 사용 가능 확인
- torch.device()로 장치 이름 가져오고, .to(device)를 사용해 향후 초기화되는 모든 텐서를 target 장치로 이동
- GPU로 데이터를 넣고 꺼내는 작업은 비용이 많이 든다.

